### Imports

In [ ]:
import os
import pandas as pd

from mp_api.client import MPRester
from matminer.featurizers.structure.composite import JarvisCFID
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty

from dotenv import load_dotenv

In [ ]:
# Initialize Materials Project API key

load_dotenv() 
mapi_key = os.environ.get('MAPI_KEY')

### Paths Setup

In [ ]:
# Set input and output file paths according to your directory structure

input_file_path = "../data/raw/exp_dataset.xlsx"
output_file_path = "../data/processed/exp_materials_with_jarvis_and_magpie.csv"

### Load Input Data

In [ ]:
if os.path.exists(input_file_path):
    exp_data = pd.read_excel(input_file_path)
    print(f"Input data of shape {exp_data.shape}, loaded from: {input_file_path}")
else:
    print("Invalid Path, directory doesn't exists.")

### Prepare Data

1. Renaming band gap column for readability. [Optional, If you dont want to change the column name update the next occurences accordingly.]

In [ ]:
exp_data.rename(columns={'Eg (eV)': 'band_gap'}, inplace=True)
exp_data.head(3)

2. Check the materials with multi-valued compositions, and calculate mean of the compositions with more than one value.

In [ ]:
composition_counts = exp_data['composition'].value_counts()
compositions_with_multiple_records = composition_counts[composition_counts > 1]

print(compositions_with_multiple_records)

In [ ]:
exp_data = exp_data.groupby('composition')['band_gap'].mean().reset_index()
exp_data.shape

3. Filtering materials, only taking materials with the bandgap between 0 and 6.

In [ ]:
exp_data = exp_data[(exp_data['band_gap'] > 0) & (exp_data['band_gap'] < 6)]
exp_data.reset_index(drop=True, inplace=True)
exp_data.shape

4. Retrieve some additional data for the filtered materials from the Materials Project.


Note that our goal is to the retrieve the structure and some other major properties of filtered materials only for the stable materails, hence specifying the threshold for the energy_above_hull (0).

In [ ]:
dataframes_to_concat = []
unstable_materials = []
others = []

mpr = MPRester(api_key=mapi_key)

for index, row in exp_data.iterrows():
    composition = str(row['composition'])
    band_gap = row['band_gap']
    is_processed = False

    print(f"Prcoessing material number : {index}/{len(exp_data)}")

    docs = mpr.summary.search(
        formula=composition,
        energy_above_hull=(0,0),
        all_fields=False,
        fields=[
            'nelements', 'formula_pretty', 'volume', 'density', 'density_atomic',
            'structure', 'uncorrected_energy_per_atom', 'energy_per_atom',
            'formation_energy_per_atom', 'efermi', 'is_gap_direct',
            'total_magnetization', 'total_magnetization_normalized_vol',
            'total_magnetization_normalized_formula_units'
        ]
    )

    if len(docs) != 0:
        sdf = pd.DataFrame([docs[0]])
        sdf['composition'] = composition
        sdf['band_gap'] = band_gap
        dataframes_to_concat.append(sdf)
        is_processed = True
    else:
        unstable_materials.append(composition)
        is_processed = True

    if not is_processed:
        others.append(composition)

In [ ]:
print(f"Total number of stable materials: {len(dataframes_to_concat)}")
print(f"Total number of unstable materials: {len(unstable_materials)}")
print(f"Total number of corrupted materials: {len(others)}")

In [ ]:
# Creating a dataframe of stable materials

result_df = pd.concat(dataframes_to_concat, ignore_index=True)
result_df.shape

5. Transform Data 

In [ ]:
def transform_dataframe(df):
    column_names = []
    for property_name in df.iloc[0].values[:-2]:
        column_names.append(str(property_name[0]))
    column_names.append('composition')
    column_names.append('band_gap')
    
    print(f"Number of Columns: {len(column_names)}")
    
    df.columns = column_names
    
    def extract_value(tuple_value):
        return tuple_value[1]
    
    columns_to_apply = df.columns[:-2]
    df[columns_to_apply] = df[columns_to_apply].applymap(extract_value)
    
    return df

In [ ]:
trans_df = transform_dataframe(result_df.copy())
trans_df.head(3)

6. Check for Nan values and drop extra columns

In [ ]:
trans_df.drop('fields_not_requested', axis=1 , inplace=True)

In [ ]:
trans_df.isna().sum()

7. Create Descriptors

In [ ]:
# Creating Jarvis descriptors from structures

jf = JarvisCFID()

In [ ]:
jarvis_df = jf.featurize_dataframe(trans_df, col_id="structure", ignore_errors=True)

In [ ]:
# Check rows with the Nan enteries

rows_with_nan = jarvis_df[jarvis_df.isna().any(axis=1)]
num_rows_with_nan = len(rows_with_nan)
num_rows_with_nan

In [ ]:
# Dropping Nan rows

jarvis_df.dropna(inplace=True)
jarvis_df.shape

In [ ]:
# Creating Magpie descriptors from compositions

comp_df = jarvis_df.copy()
str_to_comp = StrToComposition(target_col_id='composition_pmg')
comp_df = str_to_comp.featurize_dataframe(comp_df, col_id='composition')

featurizer = ElementProperty.from_preset('magpie')
comp_df = featurizer.featurize_dataframe(comp_df, col_id='composition_pmg', ignore_errors=True)
comp_df.shape

In [ ]:
# Check for the Nan values, and drop if any.

rows_with_nan = comp_df[comp_df.isna().any(axis=1)]
num_rows_with_nan = len(rows_with_nan)
num_rows_with_nan

8. Save Processed Data

In [ ]:
comp_df.to_csv(output_file_path, index=False)
print(f"Processed data of shape {comp_df.shape}, saved at: {output_file_path}")